# **Load the trained steering and collision avoidance models**

If you haven't already run the individual data collection and model training notebooks for road following/lane detection and collision avoidance/obstacle avoidance.

In [1]:
import torchvision
import torch

# Establish steering and collision models to enable jetbot to follow the track road and avoid colliding with obstacles.
steering_model = torchvision.models.resnet18(pretrained=False)
steering_model.fc = torch.nn.Linear(512, 2)

collision_model = torchvision.models.alexnet(pretrained=False)
collision_model.classifier[6] = torch.nn.Linear(collision_model.classifier[6].in_features, 2)

In [2]:
steering_model.load_state_dict(torch.load('best_steering_model_xy.pth'))
collision_model.load_state_dict(torch.load('best_model.pth'))

<All keys matched successfully>

In [3]:
device = torch.device('cuda')
steering_model = steering_model.to(device)
collision_model = collision_model.to(device)
steering_model = steering_model.eval().half()

Import torchvision, cv2, and numpy to preprocess the steering images in the model

In [4]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

# Preprocess the individual images to steer jetbot around the track
def preprocess_steering(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

Set up the mean and standard deviation of the image array. Then define the function to preprocess the the collision avoidance images of the model.

In [5]:
mean2 = 255.0 * np.array([0.485, 0.456, 0.406])
stdev2 = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean2, stdev2)

# Preprocess the camera value to determine if the jetbot's path is obstructed
def preprocess_collision(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

# Establish An Instance of The Robot

In [6]:
# Establish instance of the jetbot to enable live demo later
from jetbot import Robot

robot = Robot()

## Create the sliders to control the Jetbot's movements around the track

The initial values for steering gain, kd, and bias are all set. For my demo I upped the speed gain to 0.4 to provide a balance of speed and control in movement around the track. I'd recommend keeping it at 0 until you hit camera.observe below to prevent an initial start-up of spinning in circles.

In [7]:
# Create slider widgets to manipulate the jetbot's wheel speed, steering speed, steering drag, and steering bias
import ipywidgets

# Starting values intended for my built from scratch jetbot
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.0, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.06, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.07, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.01, description='steering bias')

# Create slider widget to display percentage of jetbot path blocked
blocked_slider = ipywidgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

# Display all recently created slider instances
display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider,blocked_slider)

FloatSlider(value=0.4, description='speed gain', max=1.0, step=0.01)

FloatSlider(value=0.06, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.07, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.01, description='steering bias', max=0.3, min=-0.3, step=0.01)

FloatSlider(value=0.0, description='blocked', max=1.0, orientation='vertical')

In [8]:
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
left_motor = ipywidgets.FloatText(description='left motor')
right_motor = ipywidgets.FloatText(description='right motor')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

## Define the execution of both thelane detection and obstacle avoidance models

In [9]:
angle = 0.0
angle_last = 0.0
left_motor_value = 0.0
right_motor_value = 0.0
from jetbot import Camera, bgr8_to_jpeg
import time

camera = Camera(fps=5)


def execute(change):
    global angle, angle_last, blocked_slider, robot
    image = change['new']
    xy = steering_model(preprocess_steering(image)).detach().float().cpu().numpy().flatten()
    pred = collision_model(preprocess_collision(image))
    prob_blocked = 0
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    pred = F.softmax(pred, dim=1)
    
    prob_blocked = float(pred.flatten()[0])
    
    blocked_slider.value = prob_blocked
    
            
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0

    x_slider.value = x
    y_slider.value = y

    speed_slider.value = speed_gain_slider.value

    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle

    steering_slider.value = pid + steering_bias_slider.value
    left_motor_value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    right_motor_value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
    left_motor.value = left_motor_value
    right_motor.value = right_motor_value

    # Check if jetbot path is blocked. If 70% blockage stop jetbot until path is clear else maintain speed
    if prob_blocked > 0.7:
        robot.stop()
    else:
        robot.left_motor.value = left_motor_value
        robot.right_motor.value = right_motor_value
        
    
    #time.sleep(0.001)
    
execute({'new': camera.value})

Display the current camera feed from the robot to show robot's path as it moves.

In [11]:
from IPython.display import display

import traitlets


image_widget = ipywidgets.Image()

def display_xy(camera_image):
    image = np.copy(camera_image)
    x = x_slider.value
    y = y_slider.value
    x = int(x * 224 / 2 + 112)
    y = int(y * 224 / 2 + 112)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (112, 224), 8, (0, 0,255), 3)
    image = cv2.line(image, (x,y), (112,224), (255,0,0), 3)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

# camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=display_xy)

display(image_widget, left_motor, right_motor)

Image(value=b'')

FloatText(value=0.5593539379131076, description='left motor')

FloatText(value=0.04064606208689231, description='right motor')

## Have the Robot update its path based on what it sees

*WARNING: The next code block will initiate movement for the jetbot if speed_gain is not 0. Be prepared for it.*

In [ ]:
camera.observe(execute, names='value')

In [12]:
import time

camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()